<a href="https://colab.research.google.com/github/osgeokr/geokakao/blob/main/kakao_local_api/Kakao_%EB%A1%9C%EC%BB%AC_API%EB%A5%BC_%EC%9D%B4%EC%9A%A9%ED%95%9C_%EA%B3%B5%EA%B0%84%EB%8D%B0%EC%9D%B4%ED%84%B0_%EA%B2%80%EC%83%89_%EA%B8%B0%EB%8A%A5_%EC%86%8C%EA%B0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kakao 로컬 API를 이용한 공간데이터 검색 기능 소개

이번 코드 실습은 [Kakao 로컬 API](https://developers.kakao.com/docs/latest/ko/local/common)를 이용한 공간데이터 검색 기능을 학습해 봅니다. 공간데이터를 검색한 이후에 지도로 가시화하는 영역은 [ipyleaflet](https://ipyleaflet.readthedocs.io/en/latest/)을 통해 구현해 보겠습니다. 먼저 필요한 라이브러리들을 불러온 후, [Kakao Developers](https://developers.kakao.com/)에서 인증받은 REST API 키를 지정합니다.
> Note: REST API는 Representational State Transfer Application Programming Interface의 약자로, 네트워크 기반 소프트웨어 시스템 간에 자원을 교환하기 위한 소프트웨어 인터페이스를 의미합니다. REST API는 웹 서비스에서 자주 사용되며, 클라이언트와 서버 간의 통신을 위한 표준적인 방법을 제공합니다.

In [ ]:
import requests
import geopandas as gpd
from ipyleaflet import Map, TileLayer, Marker, GeoData, Circle, Rectangle

In [ ]:
api_key = 'my-api-key'

## 키워드로 좌표 검색하기

특정 키워드로 해당 좌표를 검색해볼 수 있습니다. 예를 들면, '서울시립대학교'를 키워드로 해당 좌표를 검색해서 반환할 수 있습니다. 여기서는 검색 결과가 여러 개일 때는 첫번째 검색 결과의 좌표만 반환하도록 설정하였습니다.

In [ ]:
def get_coordinate(keyword, api_key=api_key):
    # 키워드로 좌표 검색하기
    url = f'https://dapi.kakao.com/v2/local/search/keyword.json?query={keyword}'
    headers = {'Authorization': f'KakaoAK {api_key}'}
    response = requests.get(url, headers=headers)
    data = response.json()
    if data['meta']['total_count'] > 0:
        # 첫 번째 검색 결과의 좌표만 반환
        return float(data['documents'][0]['y']), float(data['documents'][0]['x'])
    else:
        return None

keyword = "서울시립대학교"
coordinate = get_coordinate(keyword)
print(coordinate)

(37.584828300773886, 127.05773316246166)


In [ ]:
# Vworld 백지도 객체
vworld_white = TileLayer(
    url='https://xdworld.vworld.kr/2d/white/service/{z}/{x}/{y}.png',
    name='Vworld White',
    attribution='Vworld'
)

# 지도 생성 (Vworld 백지도 사용)
m = Map(center=coordinate,
        zoom=16,
        layers=[vworld_white],
        layout={'width': '800px', 'height': '500px'})

marker = Marker(location=coordinate, draggable=False)
m.add_layer(marker)
m

Map(center=[37.584828300773886, 127.05773316246166], controls=(ZoomControl(options=['position', 'zoom_in_text'…

## 정해진 반경에서 질의로 장소 검색하기

지정한 좌표로부터 일정 반경으로 버퍼를 설정하고, 해당 범위 내에서 장소를 검색해볼 수 있습니다. 예를 들면 앞서 검색된 서울시립대학교 위치를 중앙좌표로 하여 20km 버퍼를 적용한 후, 해당 지역 내 교보문고 매장을 검색해볼 수 있습니다. 참고로 반경의 최대값은 20km이며 튜플은 한번에 최대 45개까지 제공됩니다.

In [ ]:
def get_places_by_query(coordinate, radius, query, api_key):
    # 정해진 반경에서 이름으로 장소 검색
    url = "https://dapi.kakao.com/v2/local/search/keyword.json"
    headers = {"Authorization": f"KakaoAK {api_key}"}
    params = {"query": query,
              "x": coordinate[1],
              "y": coordinate[0],
              "radius": radius}

    places = []
    while True:
        response = requests.get(url, headers=headers, params=params)
        data = response.json()
        places.extend(data.get("documents", []))
        if data["meta"]["is_end"]:
            break
        else:
            params["page"] = params.get("page", 1) + 1
    return gpd.GeoDataFrame(
        places,
        geometry=gpd.points_from_xy(
            [place["x"] for place in places], [place["y"] for place in places]
        ),
    )

radius = 20000
query = "교보문고"

places = get_places_by_query(coordinate, radius, query, api_key)
print(f"매장 개수: {len(places)}개")
places.head(1)

매장 개수: 45개


,address_name,category_group_code,category_group_name,category_name,distance,id,phone,place_name,place_url,road_address_name,x,y,geometry
0,서울 동대문구 전농동 591-53,,,"문화,예술 > 도서 > 서점 > 교보문고",891,253947105,1544-1900,교보문고 청량리점,http://place.map.kakao.com/253947105,서울 동대문구 왕산로 214,127.049079477084,37.5806966564292,POINT (127.04908 37.58070)


교보문고를 검색했지만 모든 검색결과가 교보문고 매장을 가리키는 것은 아닙니다. 따라서 일부 컬럼에 필터링을 적용해서 원하는 튜플만을 선택해 볼 수 있습니다. 그리고 이것을 GPKG와 같은 공간데이터 포맷으로 저장하여 QGIS와 같은 다른 SW에서 사용하는 것도 가능하겠습니다.

In [ ]:
# 필터링
places = places[
    places["category_name"] == "문화,예술 > 도서 > 서점 > 교보문고"
]
print(f"매장 개수: {len(places)}개")
places = places.set_crs(epsg=4326)
places.to_file("places_by_query.gpkg", driver="GPKG")
places.head(1)

매장 개수: 18개


,address_name,category_group_code,category_group_name,category_name,distance,id,phone,place_name,place_url,road_address_name,x,y,geometry
0,서울 동대문구 전농동 591-53,,,"문화,예술 > 도서 > 서점 > 교보문고",891,253947105,1544-1900,교보문고 청량리점,http://place.map.kakao.com/253947105,서울 동대문구 왕산로 214,127.049079477084,37.5806966564292,POINT (127.04908 37.58070)


검색결과는 GeoData 객체로 변환합니다.

In [ ]:
# GeoPandas 레이어를 GeoData로 변환
geo_data = GeoData(
    geo_dataframe=places,
    point_style={
        'radius': 7,  # 점 크기
        'color': 'darkblue',  # 색상
        'fillOpacity': 0.7,  # 투명도
        'fillColor': 'lightblue',  # 점 내부 색상
        'weight': 2  # 점 테두리
    },
    name='교보문고'  # 레이어 이름
)

검색 반경도 Circle로 생성해 보겠습니다.

In [ ]:
# Circle 생성
circle = Circle()
circle.location = coordinate  # 원의 중심 좌표
circle.radius = radius  # 원의 반경
circle.color = "red"  # 원의 테두리 색상
circle.fill_color = "red"  # 원의 내부 색상
circle.fill_opacity = 0.5  # 원의 내부 투명도

이제 지도를 생성합니다. 검색 반경 내에 교보문고 매장 위치를 확인하실 수 있습니다.

In [ ]:
# 지도 생성 (Vworld 백지도 사용)
m = Map(center=coordinate,
        zoom=11,
        layers=[vworld_white],
        layout={'width': '800px', 'height': '500px'})

m.add_layer(marker)
m.add_layer(circle)
m.add_layer(geo_data)
m

Map(center=[37.584828300773886, 127.05773316246166], controls=(ZoomControl(options=['position', 'zoom_in_text'…

## 정해진 반경에서 카테고리로 장소 검색하기

카카오 로컬 API는 정해진 반경 내에서 카테고리로 장소를 검색해볼 수 있습니다. 대형마트, 편의점, 어린이집, 유치원, 학교 등 다양한 카테고리로 확인이 가능합니다. 여기서는 서울시립대학교로부터 반경 500m 이내 어린이집, 유치원을 검색해 보겠습니다.

| 이름 | MT1 | CS2 | PS3 | SC4 | AC5 | PK6 | OL7 | SW8 | BK9 | CT1 | AG2 | PO3 | AT4 | AD5 | FD6 | CE7 | HP8 | PM9 |
|------|------|------|------|------|------|------|------|------|------|------|------|------|------|------|------|------|------|------|
| 설명 | 대형마트 | 편의점 | 어린이집, 유치원 | 학교 | 학원 | 주차장 | 주유소, 충전소 | 지하철역 | 은행 | 문화시설 | 중개업소 | 공공기관 | 관광명소 | 숙박 | 음식점 | 카페 | 병원 | 약국 |

In [ ]:
def get_places_by_category(coordinate, radius, category_group_code, api_key):
    # 정해진 반경에서 카테고리로 장소 검색
    url = "https://dapi.kakao.com/v2/local/search/category.json"
    headers = {"Authorization": f"KakaoAK {api_key}"}
    params = {"category_group_code": category_group_code,
              "x": coordinate[1],
              "y": coordinate[0],
              "radius": radius}

    places = []
    while True:
        response = requests.get(url, headers=headers, params=params)
        data = response.json()
        places.extend(data.get("documents", []))
        if data["meta"]["is_end"]:
            break
        else:
            params["page"] = params.get("page", 1) + 1
    return gpd.GeoDataFrame(
        places,
        geometry=gpd.points_from_xy(
            [place["x"] for place in places], [place["y"] for place in places]
        ),
    )

radius = 500
category_group_code = "PS3"  # 어린이집,유치원

places = get_places_by_category(coordinate, radius, category_group_code, api_key)
print(f"매장 개수: {len(places)}개")
places = places.set_crs(epsg=4326)
places.to_file("places_by_category.gpkg", driver="GPKG")
places.head(1)

매장 개수: 8개


,address_name,category_group_code,category_group_name,category_name,distance,id,phone,place_name,place_url,road_address_name,x,y,geometry
0,서울 동대문구 휘경동 302-2,PS3,"어린이집,유치원","교육,학문 > 유아교육 > 유치원",182,10487569,02-2242-0354,서울휘경유치원,http://place.map.kakao.com/10487569,서울 동대문구 망우로6길 48,127.057072023037,37.586380132083,POINT (127.05707 37.58638)


In [ ]:
# GeoPandas 레이어를 GeoData로 변환
geo_data = GeoData(
    geo_dataframe=places,
    point_style={
        'radius': 7,  # 점 크기
        'color': 'darkblue',  # 색상
        'fillOpacity': 0.7,  # 투명도
        'fillColor': 'lightblue',  # 점 내부 색상
        'weight': 2  # 점 테두리
    },
    name='어린이집, 유치원'  # 레이어 이름
)

# Circle 생성
circle = Circle()
circle.location = coordinate  # 원의 중심 좌표
circle.radius = radius  # 원의 반경
circle.color = "red"  # 원의 테두리 색상
circle.fill_color = "red"  # 원의 내부 색상
circle.fill_opacity = 0.5  # 원의 내부 투명도

# 지도 생성 (Vworld 백지도 사용)
m = Map(center=coordinate,
        zoom=16,
        layers=[vworld_white],
        layout={'width': '800px', 'height': '500px'})

m.add_layer(marker)
m.add_layer(circle)
m.add_layer(geo_data)
m

Map(center=[37.584828300773886, 127.05773316246166], controls=(ZoomControl(options=['position', 'zoom_in_text'…

## 사각형 범위 내에서 제한 검색하기

검색 반경을 사각형 범위로 바꾸는 것도 가능합니다. 좌측 X 좌표, 좌측 Y 좌표, 우측 X 좌표, 우측 Y 좌표 형식으로 조회하면 됩니다.

In [ ]:
def get_places_by_category_with_rect(bbox, category_group_code, api_key):
    # 사각형 영역에서 카테고리로 장소 검색
    url = "https://dapi.kakao.com/v2/local/search/category.json"
    headers = {"Authorization": f"KakaoAK {api_key}"}

    params = {"category_group_code": category_group_code,
              "rect": bbox}

    places = []
    while True:
        response = requests.get(url, headers=headers, params=params)
        data = response.json()
        places.extend(data.get("documents", []))
        if data["meta"]["is_end"]:
            break
        else:
            params["page"] = params.get("page", 1) + 1

    return gpd.GeoDataFrame(
        places,
        geometry=gpd.points_from_xy(
            [float(place["x"]) for place in places], [float(place["y"]) for place in places]
        ),
    )

In [ ]:
# 좌측 X 좌표, 좌측 Y 좌표, 우측 X 좌표, 우측 Y 좌표 형식
bbox = "127.03506145225234,37.5938114125238,127.08040487267098,37.57584518902397"

category_group_code = "SC4" # 학교

places = get_places_by_category_with_rect(bbox, category_group_code, api_key)
print(f"매장 개수: {len(places)}개")
places = places.set_crs(epsg=4326)
places.to_file("places_by_category_with_rect.gpkg", driver="GPKG")
places.head(1)

매장 개수: 33개


,address_name,category_group_code,category_group_name,category_name,distance,id,phone,place_name,place_url,road_address_name,x,y,geometry
0,서울 동대문구 전농동 694,SC4,학교,"교육,학문 > 학교 > 대학교",,10498806,02-6490-6114,서울시립대학교,http://place.map.kakao.com/10498806,서울 동대문구 서울시립대로 163,127.05773316246166,37.584828300773886,POINT (127.05773 37.58483)


In [ ]:
# GeoPandas 레이어를 GeoData로 변환
geo_data = GeoData(
    geo_dataframe=places,
    point_style={
        'radius': 7,  # 점 크기
        'color': 'darkblue',  # 색상
        'fillOpacity': 0.7,  # 투명도
        'fillColor': 'lightblue',  # 점 내부 색상
        'weight': 2  # 점 테두리
    },
    name='학교'  # 레이어 이름
)

# Rectangle(사각형) 생성
bbox_values = [float(val) for val in bbox.split(",")]

# ((최소 위도, 최소 경도), (최대 위도, 최대 경도))
bbox_bounds = ((bbox_values[1], bbox_values[0]), (bbox_values[3], bbox_values[2]))
rectangle = Rectangle(bounds=bbox_bounds, color="red", fill_opacity=0.2)

# bbox_bounds에서 중앙 좌표 계산
center_lat = (bbox_bounds[0][0] + bbox_bounds[1][0]) / 2
center_lon = (bbox_bounds[0][1] + bbox_bounds[1][1]) / 2

# 지도 생성 (Vworld 백지도 사용)
m = Map(center=coordinate,
        zoom=14,
        layers=[vworld_white],
        layout={'width': '800px', 'height': '500px'})

m.add_layer(marker)
m.add_layer(rectangle)
m.add_layer(geo_data)
m

Map(center=[37.584828300773886, 127.05773316246166], controls=(ZoomControl(options=['position', 'zoom_in_text'…

검색하고자 하는 영역이 넓은 경우에는 전체 영역을 적정하게 분할하여 데이터를 수집한 후, 중복성을 제거하는 방식으로 병합하면 될 것 같습니다.

In [9]:
from ipyleaflet import Heatmap, Map

# 예시 위치 데이터
locations = [(37.77, -122.42, 1), (37.76, -122.43, 2), (37.75, -122.44, 3)]

# Heatmap 레이어 생성, gradient 옵션에 RGBA 색상 코드 포함
heatmap = Heatmap(
    locations=marker,
    radius=20000,
    gradient={
        0.2: 'rgba(0, 0, 255, 0.4)', # 푸른색, 낮은 투명도
        0.4: 'rgba(0, 255, 255, 0.4)', # 청록색, 중간 투명도
        0.6: 'rgba(0, 255, 0, 0.4)', # 녹색, 높은 투명도
        0.8: 'rgba(255, 255, 0, 0.4)', # 노란색, 더 높은 투명도
        1.0: 'rgba(255, 0, 0, 0.4)' # 빨간색, 투명도 없음
    }
)

# 지도 생성 및 Heatmap 레이어 추가
m = Map(center=(37.76, -122.45), zoom=12)
m.add_layer(heatmap)

m

NameError: name 'marker' is not defined

In [11]:
from ipyleaflet import Map, Heatmap
from random import uniform
m = Map(center=(0, 0), zoom=2)

heatmap = Heatmap(
    locations=[[uniform(-80, 80), uniform(-180, 180), uniform(0, 1000)] for i in range(1000)],
    radius=50
)

m.add(heatmap);

m

Map(center=[0, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text'…